### *Module Loading*

In [ ]:
import sys
import faiss
from subprocess import PIPE, run
from IPython.display import display as ip_display

### *External Module Loading*

In [ ]:
external_modules_path = '..\\nn_likelihood_modules'
sys.path.append(external_modules_path)

In [ ]:
from basic_network_structure import *
from common_imports import *
from common_use_functions import *
from constant import *
from defined_data_structure import *
from defined_network_structure import *
from experim_neural_network import *
from experim_preparation import *
from generate_activation_level import *
from pytorch_model_predict import *
from vector_preprocessing import *
from ResNet import *
from experim_ResNet import *
from cifar_10_data_prep import *
from sensitivity_analysis import *
from deep_KNN import *
from novelty_data_prep import *
from activation_level_processing import *

### *GPU verification*

In [ ]:
# Get the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nb_gpu = torch.cuda.device_count()
if nb_gpu > 0:
    print(torch.cuda.get_device_name(0))
else:
    print("CPU")

### *Working directory*

In [ ]:
# Current path
current_path = os.path.abspath(os.getcwd())

### *Load configurations and data*

In [ ]:
"""
All the parameters in this part should be configured
"""
# Experience path
experim_path = current_path

# File extensions
json_ext = '.json'
np_ext = '.npy'
csv_ext = '.csv'

# ResNet model prefix
model_name_prefix = 'cifar10'

# Image max pixel value
image_max_pix_val = 255

# Tested sets name
train_set_name = 'train'
test_set_name = 'test'
valid_set_name = 'valid'
input_extension = 'X'
label_extension = 'Y'

# Save paths
model_save_path = path_join(experim_path, 'experim_models_resnet')

# Adversarial attack path
adv_attack_path = path_join(experim_path, 'experim_resnet_attack')

"""
The following parameters should be configured according to your experiments
"""

# Trained model name 
trained_resnet_name = 'cifar10_resnet50_938' # You can select any model from the "experim_models_resnet" folder

# Indices filenames
train_indices_filename = 'train_indices_938' # The train indices should be coherent with your chosen model
valid_indices_filename = 'valid_indices_938' # The valid indices should be coherent with your chosen model

# ResNet related params
resnet_model_name = 'resnet50'# The model name should be coherent with your chosen model

# Cifar10-c datafolder path
cifar10_c_path = 'D:\\Doctorat\\research\\oms_detection_experim\\CIFAR-10-C\CIFAR-10-C\\'

# Dataset general informations
data_set_infos = {
    'nb_classes' : 10
}

# Distance decision filtering threshold
std_threshold_coeff = 2

# The method to determining the significant neurons (mean or most_common)
sobol_filter_method = 'most_common'

# The divide factor used for "top portion" and "beside end portion" methods which determine the significant neurons
sobol_divide_factor = 4

# Rscript launch params
Rscript_path = 'C:\\Program Files\\R\\R-4.3.3\\bin\\Rscript.exe'

# The number of considered k-nearst neighbors
k = 50

"""
"""

# The output folder
output_path = path_join(experim_path, 'output')

# Build the class list
class_list = list(range(data_set_infos['nb_classes']))

# Batch size for the dataloader creation
torch_batch_size = 128

In [ ]:
# Create the folders
create_directory(output_path)

### *Experiment preparation*

In [ ]:
# Get the dataset
cifar10_train_dataset, cifar10_test_dataset = get_cifar10_dataset_without_transform()

In [ ]:
# The column names of the filtering results
column_names_OOD_filtering = ['transformation', 'nb_examples', 'nb_OOD', 'nb_InD', 'total_acc', 'OOD_acc', 'InD_acc']

### *Load the trained ResNet*

In [ ]:
# Create the resnet
trained_resnet = load_model_by_net_name(model_save_path, trained_resnet_name)

### *Move the model to GPU*

In [ ]:
# Move to gpu
trained_resnet.cuda()

### *Cifar10 dataset preparation*

In [ ]:
# Load train valid indices
train_indices = load_json(open(path_join(model_save_path, train_indices_filename+json_ext)))
valid_indices = load_json(open(path_join(model_save_path, valid_indices_filename+json_ext)))

In [ ]:
# Build the subsets
cifar10_real_train_dataset = Subset(cifar10_train_dataset, train_indices)
cifar10_valid_dataset = Subset(cifar10_train_dataset, valid_indices)

In [ ]:
# Dataloader building
train_loader = create_loader_from_torch_dataset(cifar10_real_train_dataset, batch_size=torch_batch_size, shuffle=False, num_workers=0)
valid_loader = create_loader_from_torch_dataset(cifar10_valid_dataset, batch_size=torch_batch_size, shuffle=False, num_workers=0)
test_loader = create_loader_from_torch_dataset(cifar10_test_dataset, batch_size=torch_batch_size, shuffle=False, num_workers=0)

In [ ]:
# Convert the training set to numpy array
no_divide_into_batch_train_loader = create_loader_from_torch_dataset(cifar10_real_train_dataset, batch_size=len(cifar10_real_train_dataset), shuffle=False, num_workers=0)
cifar10_train_X = next(iter(no_divide_into_batch_train_loader))[0].numpy()
cifar10_train_y = next(iter(no_divide_into_batch_train_loader))[1].numpy()

In [ ]:
# Convert the test set to numpy array
no_divide_into_batch_test_loader = create_loader_from_torch_dataset(cifar10_test_dataset, batch_size=len(cifar10_test_dataset), shuffle=False, num_workers=0)
X_test = next(iter(no_divide_into_batch_test_loader))[0].numpy()
y_test = next(iter(no_divide_into_batch_test_loader))[1].numpy()

### *Evaluate the activation levels for the original training and test sets of CIFAR-10*

In [ ]:
# Get the training set activation levels 
train_actLevels = obtain_activation_levels(trained_resnet,
                                           train_loader, 'train', with_predict_class=True, loss_type='cross_entropy')

In [ ]:
# Get the test set activation levels 
test_actLevels = obtain_activation_levels(trained_resnet,
                                           test_loader, 'test', with_predict_class=True, loss_type='cross_entropy')

### *Sobol index evaluation*

In [ ]:
# Last hidden layer Id
last_hidden_layerId = list(train_actLevels['actLevel'].keys())[-1]

In [ ]:
# Get the correctly predicted index
correctly_predicted_bools = train_actLevels['class'] == train_actLevels['predict_class']
# Get the corresponding data
last_hidden_actLevels = train_actLevels['actLevel'][last_hidden_layerId][correctly_predicted_bools.reshape(-1)]

In [ ]:
# Get the last layer parameters
model_params = get_model_parameters(trained_resnet, to_numpy=True)
final_linear_params = model_params['linear_out']

In [ ]:
# Normalization of weight and input
# Copy the original activation levels and weights
normalized_last_hidden_actLevel = copy.deepcopy(last_hidden_actLevels)
upped_final_linear_params = copy.deepcopy(final_linear_params)
# Check the min and max values of each neuron
last_hidden_actLevel_max = np.max(last_hidden_actLevels, axis=0)
last_hidden_actLevel_min = np.min(last_hidden_actLevels, axis=0) # Not used, just for verification
# Iterate over the maximum values and normalize the input
for index, neuron_max in enumerate(last_hidden_actLevel_max):
    if neuron_max != 0:
        normalized_last_hidden_actLevel[:,index] = normalized_last_hidden_actLevel[:,index] / neuron_max
        upped_final_linear_params['weight'][:,index] = upped_final_linear_params['weight'][:,index] * neuron_max

In [ ]:
# Build the final linear parameters per class and assign the real data
used_linear_params = upped_final_linear_params
data = normalized_last_hidden_actLevel
# Number of variables
nb_vars = data.shape[1]

In [ ]:
## Build the X and y for the sobol index evaluation in R
# Get the neuron names
neuron_names = ['neuron_'+str(index) for index in range(last_hidden_actLevels.shape[1])]
# Build the X dataframe
R_X = pd.DataFrame(data, columns=neuron_names)

In [ ]:
# Build the dataframe that stores weights and bias
R_network_params_weight_columns = [*(['weight_'+str(index) for index in range(used_linear_params['weight'].shape[1])])]
R_network_params_bias_columns = [*(['bias_'+str(index) for index in range(used_linear_params['bias'].shape[0])])]
R_network_params_weights = pd.DataFrame(used_linear_params['weight'], columns=R_network_params_weight_columns)
R_network_params_bias = pd.DataFrame(used_linear_params['bias'].reshape(1,-1), columns=R_network_params_bias_columns)
save_df_to_csv(path_join(output_path, 'R_network_params_weight.csv'),R_network_params_weights)
save_df_to_csv(path_join(output_path, 'R_network_params_bias.csv'),R_network_params_bias)

In [ ]:
# Sample size
N = 10000

In [ ]:
# Generate two random samples from R_X
A_index = generate_sample_index(data, N, replace=False)
B_index = generate_sample_index_exclude_items(data, N, A_index, replace=False)
R_X_A = R_X.loc[A_index,:].copy(deep=True).reset_index(drop=True)
R_X_B = R_X.loc[B_index,:].copy(deep=True).reset_index(drop=True)
# Save the random samples from R_X
save_df_to_csv(path_join(output_path, 'R_X_A.csv'),R_X_A)
save_df_to_csv(path_join(output_path, 'R_X_B.csv'),R_X_B)

In [ ]:
# You could choose sobolrank (only first order indices), sobolEff (first and total indices) or shapleysobol_knn (first and total indices)
R_sobol_method = 'sobolmultout'
R_sobol_script = path_join(experim_path, R_sobol_method+'_eval.R')

In [ ]:
# Execute the R script (The path to read the data is given as arguments (i.e., experim_path+'\\'))
ouput_R = run([Rscript_path, '--vanilla', R_sobol_script, output_path+'\\', str(data_set_infos['nb_classes'])], shell=True) 

In [ ]:
## Get the important variables per class
# Initialize the determined variables as None
important_variables = None
if R_sobol_method == 'sobolmultout':
    # Load the sobol indices (first and total) and convert it to a dictionary
    R_first_order_sobol_indices = read_csv_to_pd_df(path_join(output_path, R_sobol_method+'_fs'+csv_ext))
    R_total_order_sobol_indices = read_csv_to_pd_df(path_join(output_path, R_sobol_method+'_tt'+csv_ext))
    R_first_order_sobol_dict = R_first_order_sobol_indices['original'].to_dict()
    R_total_order_sobol_dict = R_total_order_sobol_indices['original'].to_dict()
    # Get the important variables per class
    important_variables = important_variables_R_first_and_total_order_analysis_multout_ver(R_first_order_sobol_dict, 
                                                                                           R_total_order_sobol_dict,
                                                                                           filter_method=sobol_filter_method,
                                                                                           divide_factor=4)

In [ ]:
## Build the dataframe that contains the number of neurons
nb_important_vars = len(list(important_variables.keys()))
determined_nb_important_vars = None
if sobol_filter_method != 'top_portion' and sobol_filter_method != 'beside_end_portion':
    determined_nb_important_vars = [[nb_vars, nb_important_vars, sobol_filter_method]]
else:
    determined_nb_important_vars = [[nb_vars, nb_important_vars, sobol_filter_method+'_'+sobol_divide_factor]]
determined_nb_important_vars_df = pd.DataFrame(determined_nb_important_vars, columns=['nb_neurons', 'nb_important_neurons', 'sobol_filter_method'])

In [ ]:
# Build the sorted total important variable(neuron) indices
sorted_important_vars = sorted(list(important_variables))
# Build the mapping dictionary to modify neuron indices
important_var_map_dict = build_map_to_index_dict(important_variables)

In [ ]:
# Save the important variables
store_dict_as_json(path_join(output_path, trained_resnet_name+'_important_neurons.json'), important_variables)
save_df_to_csv(path_join(output_path, trained_resnet_name+'_nb_important_neurons.csv'), determined_nb_important_vars_df)

### *Load the novelty OOD dataset and evaluate the activation levels*

In [ ]:
# Get the svhn dataset
svhn_train_dataset, svhn_test_dataset = get_svhn_dataset_without_transform()

In [ ]:
# Dataloader building
# svhn_train_loader = create_loader_from_torch_dataset(svhn_train_dataset, batch_size=torch_batch_size, shuffle=False, num_workers=0)
svhn_test_loader = create_loader_from_torch_dataset(svhn_test_dataset, batch_size=torch_batch_size, shuffle=False, num_workers=0)

In [ ]:
# Get the dtd dataset
dtd_train_dataset, dtd_test_dataset = get_dtd_dataset_resized()

In [ ]:
# # Convert the train set to numpy array
# no_divide_into_batch_dtd_train_loader = create_loader_from_torch_dataset(dtd_train_dataset, batch_size=len(dtd_train_dataset), shuffle=False, num_workers=0)
# X_train_dtd = next(iter(no_divide_into_batch_dtd_train_loader))[0].numpy()
# y_train_dtd = next(iter(no_divide_into_batch_dtd_train_loader))[1].numpy()

In [ ]:
# Convert the test set to numpy array
no_divide_into_batch_dtd_test_loader = create_loader_from_torch_dataset(dtd_test_dataset, batch_size=len(dtd_test_dataset), shuffle=False, num_workers=0)
X_test_dtd = next(iter(no_divide_into_batch_dtd_test_loader))[0].numpy()
y_test_dtd = next(iter(no_divide_into_batch_dtd_test_loader))[1].numpy()

In [ ]:
# Build the dtd loaders (using random original labels (because they are not important))
# dtd_train_loader = create_dataloader(X_train_dtd, np.random.randint(0, data_set_infos['nb_classes'], y_train_dtd.shape[0]), 
#                                      batch_size=torch_batch_size, shuffle=False, type_conversion=True)
dtd_test_loader = create_dataloader(X_test_dtd, np.random.randint(0, data_set_infos['nb_classes'], y_test_dtd.shape[0]), 
                                     batch_size=torch_batch_size, shuffle=False, type_conversion=True)

In [ ]:
# Get the places365 dataset
places_test_dataset = get_places_test_dataset_resized()

In [ ]:
# Convert the test set to numpy array
no_divide_into_batch_places_test_loader = create_loader_from_torch_dataset(places_test_dataset, batch_size=len(places_test_dataset), shuffle=False, num_workers=0)
X_test_places = next(iter(no_divide_into_batch_places_test_loader))[0].numpy()
y_test_places = next(iter(no_divide_into_batch_places_test_loader))[1].numpy()

In [ ]:
# Build the places365 loaders (using random original labels (because they are not important))
places_test_loader = create_dataloader(X_test_places, np.random.randint(0, data_set_infos['nb_classes'], y_test_places.shape[0]), 
                                     batch_size=torch_batch_size, shuffle=False, type_conversion=True)

In [ ]:
# Build the dictionary that contains all the OOD dataset loaders
novelty_loaders = {}
novelty_loaders['svhn'] = svhn_test_loader
novelty_loaders['dtd'] = dtd_test_loader
novelty_loaders['places'] = places_test_loader

In [ ]:
# Iterate over the OMS datasets for generating the normalized feature vectors
novelty_actLevels = {}
for ood_type in novelty_loaders:
    novelty_actLevels[ood_type] = obtain_activation_levels(trained_resnet,
                                                           novelty_loaders[ood_type], ood_type + ' test',
                                                           with_predict_class=True, loss_type='cross_entropy')

### *Load the distribution shift OOD dataset (CIFAR-10-c) and evaluate the activation levels*

In [ ]:
## Read the cifar10-c dataset
# Get the content in the folder
cifar10_c_data_files = [file for file in contents_of_folder(cifar10_c_path) if np_ext in file]
# The number of images at each level
nb_image_by_level = 10000
# Load all the files
load_cifar10_c = {}
for file in cifar10_c_data_files:
    # We always use the transformation_type as variable name for the ease of coding (even if it could be just "labels")  
    file_type = str_first_part_split_from_r(file)
    current_data = np.load(path_join(cifar10_c_path, file))
    nb_batchs = current_data.shape[0] / nb_image_by_level
    for index in range(int(nb_batchs)):
        current_batch_data = current_data[index*nb_image_by_level:(index+1)*nb_image_by_level]
        load_cifar10_c[file_type+'_s'+str(index+1)] = current_batch_data

In [ ]:
# Build the dataloader and evaluate the activation levels
distrib_shift_actLevels = {}
for transformation_type in load_cifar10_c:
    if 'labels' not in transformation_type:
        # Get the severe level
        severity = str_second_part_split_from_r(transformation_type, delimiter='_')
        # Get the current image array
        transformed_image_array = load_cifar10_c[transformation_type]
        # Reshape the numpy array to satisfy pytorch model requirements     
        pytorch_transformed_image_array = transformed_image_array.transpose(0,3,1,2)
        # Normalize the pixel values to (0,1) range
        pytorch_transformed_image_array = pytorch_transformed_image_array / image_max_pix_val
        # Build the loader         
        current_transformed_loader = create_dataloader(pytorch_transformed_image_array, load_cifar10_c['labels_'+severity],
                                                                   torch_batch_size, shuffle=False, type_conversion=True)
        # Evaluate the activation levels
        distrib_shift_actLevels[transformation_type] = obtain_activation_levels(trained_resnet,
                                           current_transformed_loader, transformation_type, with_predict_class=True, loss_type='cross_entropy')

### *Load the adversarial OOD dataset and evaluate the activation levels*

In [ ]:
# The registrered original dataset for the attacks
original_X = None
original_y = None

In [ ]:
## Read the adversarial attacks
# Find the loaded trained resnet attack path
trained_resnet_attack_path = None
for attack_folder in contents_of_folder(adv_attack_path):
    if trained_resnet_name in attack_folder:
        trained_resnet_attack_path = path_join(adv_attack_path, attack_folder)
        break
# Load all the attacks
loaded_attacks = {}
for attack_set in contents_of_folder(trained_resnet_attack_path):
    current_attack_type = str_first_part_split_from_l(attack_set)
    current_attack_set_path = path_join(trained_resnet_attack_path, attack_set)
    if current_attack_type == 'original':
        X_file_path = path_join(current_attack_set_path, 'X.npy')
        y_file_path = path_join(current_attack_set_path, 'y.npy')
        original_X = np.load(X_file_path)
        original_y = np.load(y_file_path)
    else:
        attack_file_path = path_join(current_attack_set_path, contents_of_folder(current_attack_set_path)[0])
        loaded_attacks[current_attack_type] = np.load(attack_file_path)

In [ ]:
# Build the attack loaders
attack_loaders = {}
for attack_type in loaded_attacks:
    attack_loaders[attack_type] = create_dataloader(loaded_attacks[attack_type], original_y, torch_batch_size, shuffle=False, type_conversion=True)

In [ ]:
# Evaluate the attacks
for attack_type in attack_loaders:
    accuracy_eval(trained_resnet, attack_loaders[attack_type], set_name=attack_type)

In [ ]:
# Evaluate the activation levels
attack_actLevels = {}
for attack_type in attack_loaders:
    attack_actLevels[attack_type] = obtain_activation_levels(trained_resnet,
                                       attack_loaders[attack_type], attack_type, with_predict_class=True, loss_type='cross_entropy')

### *Determine the correctly predicted training examples*

In [ ]:
# Boolean indicating the correctly predicted training examples
correct_train_actLevels = build_correct_actLevels(train_actLevels)

### *Prepare the normalized feature vectors with all neurons*

In [ ]:
# Build the training set normalized feature vectors (only the correctly predicted examples)
correct_train_zs = normalize_feature_vecs_knn(correct_train_actLevels, last_hidden_layerId)

In [ ]:
# Build the training set normalized feature vectors
train_zs = normalize_feature_vecs_knn(train_actLevels, last_hidden_layerId)

In [ ]:
# Build the test set normalized feature vectors
test_zs = normalize_feature_vecs_knn(test_actLevels, last_hidden_layerId)

In [ ]:
# Get the normalized feature vectors of the novelty ood set
novelty_zs = {}
for ood_type in novelty_actLevels:
    novelty_zs[ood_type] = normalize_feature_vecs_knn(novelty_actLevels[ood_type], last_hidden_layerId)

In [ ]:
# Get the normalized feature vectors of the distribution shift ood set
distrib_shift_zs = {}
for transform_type in distrib_shift_actLevels:
    distrib_shift_zs[transform_type] = normalize_feature_vecs_knn(distrib_shift_actLevels[transform_type], last_hidden_layerId)

In [ ]:
attack_zs = {}
for attack_type in attack_actLevels:
    attack_zs[attack_type] = normalize_feature_vecs_knn(attack_actLevels[attack_type], last_hidden_layerId)

### *Prepare the Deep k-nearst neighbors OOD detection with all neurons*

In [ ]:
# Build the faiss train set index
train_index = faiss.IndexFlatL2(correct_train_zs.shape[1])
train_index.add(correct_train_zs)

In [ ]:
# Evaluate the train set k-nearst neighbor scores
correct_train_S = k_nearst_neighbor_scores(train_index, correct_train_zs, k)

In [ ]:
# Get the mean and std of the test scores
threshold_S_mean, threshold_S_std = get_mean_std(correct_train_S)

In [ ]:
# Get the threshold
threshold = threshold_S_mean - std_threshold_coeff * threshold_S_std

### *Deep k-nearst neighbors OOD evaluation result headers*

In [ ]:
knn_ood_eval_headers = ['set_name', 'nb_ood', 'nb_ind', 'ood_percent', 'ind_percent', 'acc_total', 'acc_ood', 'acc_ind']

### *Evaluate the test accuracy after applying the OOD detection with all neurons on the original dataset*

In [ ]:
# Initialize the ood results on the original datasets
origin_ood_eval_results = []
# The ood detection evaluation on the training set
train_ood_eval_result = experim_ood_detection_knn(train_index, train_zs, train_actLevels, k, threshold, 'train')
# Display jump line
print()
# The ood detection evaluation on the test set
test_ood_eval_result = experim_ood_detection_knn(train_index, test_zs, test_actLevels, k, threshold, 'test')
# Add the evaluation results
origin_ood_eval_results.append(train_ood_eval_result)
origin_ood_eval_results.append(test_ood_eval_result)

In [ ]:
# Save the train ood evaluation results
save_list_to_csv(path_join(output_path, 'origin_ood_eval_results'+csv_ext), origin_ood_eval_results, headers=knn_ood_eval_headers)

### *Deep k-nearst neighbors OOD detection on novelty dataset*

In [ ]:
# The ood detection evaluation
novelty_ood_eval_results = []
for ood_type in novelty_zs:
    current_ood_eval_result = experim_ood_detection_knn(train_index, novelty_zs[ood_type],
                                                        novelty_actLevels[ood_type], k, threshold, ood_type)
    novelty_ood_eval_results.append(current_ood_eval_result)
    print()

In [ ]:
# Save the distribution shift ood evaluation results
save_list_to_csv(path_join(output_path, 'novelty_ood_eval_results'+csv_ext), novelty_ood_eval_results, headers=knn_ood_eval_headers)

### *Deep k-nearst neighbors OOD detection on distribution shift dataset (CIFAR-10-c)*

In [ ]:
# The ood detection evaluation
distrib_shift_eval_results = []
for transform_type in distrib_shift_zs:
    current_eval_result = experim_ood_detection_knn(train_index, distrib_shift_zs[transform_type],
                                                    distrib_shift_actLevels[transform_type], k, threshold, transform_type)
    distrib_shift_eval_results.append(current_eval_result)
    print()

In [ ]:
# Save the distribution shift ood evaluation results
save_list_to_csv(path_join(output_path, 'distrib_shift_ood_eval_results'+csv_ext), distrib_shift_eval_results, headers=knn_ood_eval_headers)

### *Deep k-nearst neighbors OOD detection on adversarial dataset*

In [ ]:
# The ood detection evaluation
attack_eval_results = []
for attack_type in attack_zs:
    current_eval_result = experim_ood_detection_knn(train_index, attack_zs[attack_type],
                                                    attack_actLevels[attack_type], k, threshold, attack_type)
    attack_eval_results.append(current_eval_result)
    print()

In [ ]:
# Save the distribution shift ood evaluation results
save_list_to_csv(path_join(output_path, 'adversarial_ood_eval_results'+csv_ext), attack_eval_results, headers=knn_ood_eval_headers)

### *Prepare the normalized feature vectors with only the significant neurons*

In [ ]:
# Take the normalized feature vectors according to the significant neurons of each class for the search index building
correct_train_zs_sig = build_sig_zs(correct_train_actLevels, last_hidden_layerId, sorted_important_vars)

In [ ]:
# Take the normalized feature vectors according to the significant neurons of each class for the training set
train_zs_sig = build_sig_zs(train_actLevels, last_hidden_layerId, sorted_important_vars)

In [ ]:
# Take the normalized feature vectors according to the significant neurons of each class for the test set
test_zs_sig = build_sig_zs(test_actLevels, last_hidden_layerId, sorted_important_vars)

In [ ]:
# Get the normalized feature vectors of the novelty ood set using only the significant neurons
novelty_zs_sig = {}
for ood_type in novelty_actLevels:
    novelty_zs_sig[ood_type] = build_sig_zs(novelty_actLevels[ood_type], last_hidden_layerId, sorted_important_vars)

In [ ]:
# Take only the normalized feature vectors from the significant neurons for the distribution shift ood set
distrib_shift_zs_sig = {}
for transform_type in distrib_shift_actLevels:
    distrib_shift_zs_sig[transform_type] = build_sig_zs(distrib_shift_actLevels[transform_type], last_hidden_layerId, sorted_important_vars)

In [ ]:
# Take only the normalized feature vectors from the significant neurons for the adversarial ood set
attack_zs_sig = {}
for attack_type in attack_actLevels:
    attack_zs_sig[attack_type] = build_sig_zs(attack_actLevels[attack_type], last_hidden_layerId, sorted_important_vars)

### *Prepare the Deep k-nearst neighbors OOD detection with the significant neurons*

In [ ]:
# Build the faiss train set index
train_index_sig = faiss.IndexFlatL2(correct_train_zs_sig.shape[1])
train_index_sig.add(correct_train_zs_sig)

In [ ]:
# Build the scores over different classes
correct_train_S_sig = k_nearst_neighbor_scores(train_index_sig, correct_train_zs_sig, k)

In [ ]:
# Get the mean and std of the train scores
threshold_S_mean_sig, threshold_S_std_sig = get_mean_std(correct_train_S_sig)

In [ ]:
# Get the threshold
threshold_sig = threshold_S_mean_sig - std_threshold_coeff * threshold_S_std_sig

### *Evaluate the test accuracy after applying the OOD detection with the significant neurons*

In [ ]:
# Initialize the ood results on the original datasets
sig_origin_ood_eval_results = []
# The ood detection evaluation on the training set
sig_train_ood_eval_result = experim_ood_detection_knn(train_index_sig, train_zs_sig, train_actLevels, k, threshold_sig, 'train')
# Display jump line
print()
# The ood detection evaluation on the test set
sig_test_ood_eval_result = experim_ood_detection_knn(train_index_sig, test_zs_sig, test_actLevels, k, threshold_sig, 'test')
# Add the evaluation results
sig_origin_ood_eval_results.append(sig_train_ood_eval_result)
sig_origin_ood_eval_results.append(sig_test_ood_eval_result)

In [ ]:
# Save the train ood evaluation results
save_list_to_csv(path_join(output_path, 'sig_origin_ood_eval_results'+csv_ext), sig_origin_ood_eval_results, headers=knn_ood_eval_headers)

### *Deep k-nearst neighbors OOD detection with only the significant neurons on novelty dataset*

In [ ]:
# The ood detection evaluation
sig_novelty_ood_eval_results = []
for ood_type in novelty_zs_sig:
    current_eval_result = experim_ood_detection_knn(train_index_sig, novelty_zs_sig[ood_type],
                                                    novelty_actLevels[ood_type], k, threshold_sig, ood_type)
    sig_novelty_ood_eval_results.append(current_eval_result)
    print()

In [ ]:
# Save the distribution shift ood evaluation results
save_list_to_csv(path_join(output_path, 'sig_novelty_ood_eval_results'+csv_ext), sig_novelty_ood_eval_results, headers=knn_ood_eval_headers)

### *Deep k-nearst neighbors OOD detection with only the significant neurons on distribution shift dataset*

In [ ]:
# The ood detection evaluation
sig_distrib_shift_eval_results = []
for transform_type in distrib_shift_zs_sig:
    current_eval_result = experim_ood_detection_knn(train_index_sig, distrib_shift_zs_sig[transform_type],
                                                    distrib_shift_actLevels[transform_type], k, threshold_sig, transform_type)
    sig_distrib_shift_eval_results.append(current_eval_result)
    print()

In [ ]:
# Save the distribution shift ood evaluation results
save_list_to_csv(path_join(output_path, 'sig_distrib_shift_ood_eval_results'+csv_ext), sig_distrib_shift_eval_results, headers=knn_ood_eval_headers)

### *Deep k-nearst neighbors OOD detection with only the significant neurons on adversarial dataset*

In [ ]:
# The ood detection evaluation
sig_attack_eval_results = []
for attack_type in attack_zs_sig:
    current_eval_result = experim_ood_detection_knn(train_index_sig, attack_zs_sig[attack_type],
                                                    attack_actLevels[attack_type], k, threshold_sig, attack_type)
    sig_attack_eval_results.append(current_eval_result)
    print()

In [ ]:
# Save the distribution shift ood evaluation results
save_list_to_csv(path_join(output_path, 'sig_adversarial_ood_eval_results'+csv_ext), sig_attack_eval_results, headers=knn_ood_eval_headers)